In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
!wget -O models/yolov3.weights https://pjreddie.com/media/files/yolov3.weights

In [ ]:
# Load YOLO
net = cv2.dnn.readNet("../../models/yolov3.weights", "../../cfg/yolov3.cfg")
classes = []
with open("../../data/labels/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
print('Number of classes:', len(classes))

layer_names = net.getLayerNames()
output_layers_indices = [i - 1 for i in net.getUnconnectedOutLayers()]
output_layers = [layer_names[i] for i in output_layers_indices]

print("Output layers:", output_layers)